<a href="https://colab.research.google.com/github/GeorgeSere/Projects/blob/main/%D0%98%D1%81%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BD%D0%B0%D0%B4%D1%91%D0%B6%D0%BD%D0%BE%D1%81%D1%82%D0%B8_%D0%B7%D0%B0%D1%91%D0%BC%D1%89%D0%B8%D0%BA%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ТЕМПЛЕЙТ:

In [ ]:
import pandas as pd
import numpy as np
import io
import sqlite3

In [ ]:
con = sqlite3.connect('db')

In [ ]:
def select(sql):
  return pd.read_sql(sql,con)

In [ ]:
cur = con.cursor()

# О ПРОЕКТЕ:

В рамках данного проекта мне предстоит ознакомиться с данными о заёмщиках банка. Заняться предобработкой и исследованием данных с целью проверки ряда гипотез и получения ответов на вопросы, таких как:
*   Есть ли зависимость между количеством детей и возвратом кредита в срок?
*   Есть ли зависимость между семейным положением и возвратом кредита в срок?
*   Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
*   Как разные цели кредита влияют на его возврат в срок?





# ЗНАКОМСТВО С ТАБЛИЦЕЙ:

In [ ]:
from google.colab import files 
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['credits.csv']))


Saving credits.csv to credits.csv


In [ ]:
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Даже при беглом просмотре датафрейма мы замечаям ряд проблем:

*   Пропуски
*   Неявные дубликаты
*   столбец 'purpose' заполняется в свободной форме, его нужно будет категоризировать

Сейчас изучим проблемы данных основательнее


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Мы видим, что:

*   Датафрейм имеет длинну *21525* строк
*   имеет *12* различных столбцов с инофрмацией о клиентах
*   Столбцы 'days_employed' и 'total_income' имет по *2174* пропуска. Их вероятнее всего придется заменить медианными для категории занятости значениями.




# ПРЕДОБРАБОТКА ДАННЫХ:

В двух столбцах есть пропущенные значения. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца *income_type*

In [ ]:
median_income = df[df['total_income'].notna()].groupby('income_type')['total_income'].median()
for income_type, income  in zip(median_income.index, median_income):
    df.loc[(df['total_income'].isna()) & (df['income_type'] == income_type),'total_income'] = income

In [ ]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64


В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце days_employed

In [ ]:
print(df['days_employed'].unique())

[-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]


In [ ]:
df['days_employed'] = df['days_employed'].abs()

In [ ]:
print(df['days_employed'].unique())

[8437.67302776 4024.80375385 5623.42261023 ... 2113.3468877  3112.4817052
 1984.50758853]


Для каждого типа занятости выведем медианное значение трудового стажа *days_employed* в днях :

In [ ]:
print(df.groupby('income_type')['days_employed'].median())

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64


*У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.*

Выведем перечень уникальных значений столбца *children*

In [ ]:
df.to_sql('credits',con,index=False,if_exists='replace')

In [ ]:
sql = '''SELECT 
distinct t.children as children_unique

FROM credits t'''

In [ ]:
select(sql)

,children_unique
0,1
1,0
2,3
3,2
4,-1
5,4
6,20
7,5


Видим аномальные значения:
*   однозначно -1
*   скорее всего 20, тк оно идет следующим значением после 5



В столбце *children* есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения:

In [ ]:
df = df[df.children != -1]
df = df[df.children != 20]

In [ ]:
df.to_sql('credits',con,index=False,if_exists='replace')

Ещё раз выведите перечень уникальных значений столбца children, чтобы убедиться, что артефакты удалены:

In [ ]:
select(sql)

,children_unique
0,1
1,0
2,3
3,2
4,4
5,5


Отлично, всё сработало.

Заполним пропуски в столбце *days_employed* медианными значениями по каждому типу занятости *income_type* :

In [ ]:
median_days = df[df['days_employed'].notna()].groupby('income_type')['days_employed'].median()
for income_type, days  in zip(median_days.index, median_days):
    df.loc[(df['days_employed'].isna()) & (df['income_type'] == income_type),'days_employed'] = days

Убедимся, что все пропуски заполнены

In [ ]:
df.to_sql('credits',con,index=False,if_exists='replace')

In [ ]:
sql = '''SELECT 
SUM(CASE WHEN t.days_employed IS NULL THEN 1 ELSE 0 END) as days_employed_is_null

FROM credits t'''

In [ ]:
select(sql)

,days_employed_is_null
0,0


In [ ]:
print(df.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


С пропусками мы справились, двигаемся дальше.

Заменим вещественный тип данных в столбце *total_income* на целочисленный

In [ ]:
df['total_income'] = df['total_income'].astype('int')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  int64  
 11  purpose           21402 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


Обработаем неявные дубликаты в столбце *education*. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв, приведем их к нижнему регистру.

In [ ]:
df.to_sql('credits',con,index=False,if_exists='replace')

In [ ]:
sql = '''SELECT 
distinct t.education as client_id_unique

 FROM credits t'''
 
select(sql)

,client_id_unique
0,высшее
1,среднее
2,Среднее
3,СРЕДНЕЕ
4,ВЫСШЕЕ
5,неоконченное высшее
6,начальное
7,Высшее
8,НЕОКОНЧЕННОЕ ВЫСШЕЕ
9,Неоконченное высшее


In [ ]:
df['education'] = df['education'].str.lower()

In [ ]:
df.to_sql('credits',con,index=False,if_exists='replace')

In [ ]:
sql = '''SELECT 
distinct t.education as client_id_unique

 FROM credits t'''
 
select(sql)

,client_id_unique
0,высшее
1,среднее
2,неоконченное высшее
3,начальное
4,ученая степень


Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, то удалим их:

In [ ]:
df.duplicated().sum()

71

In [ ]:
df.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
df.duplicated().sum()

0

# КАТЕГОРИЗАЦИЯ ДАННЫХ:

На основании диапазонов, указанных ниже, создайте в датафрейме *data* столбец *total_income_category* с категориями:

* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

In [ ]:
df.to_sql('credits',con,index=False,if_exists='replace')

In [ ]:
sql = '''SELECT t.*,

CASE WHEN t.total_income <= 30000 THEN 'E'
WHEN t.total_income <= 50000 THEN 'D' 
WHEN t.total_income <= 200000 THEN 'C'
WHEN t.total_income <= 1000000 THEN 'B'
WHEN t.total_income > 1000000 THEN 'A'
ELSE 'other' END 
AS total_income_category

FROM credits t'''

In [ ]:
df = select(sql)

In [ ]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


Выведем на экран перечень уникальных целей взятия кредита из столбца *purpose* :

In [ ]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим на основании данных из столбца *purpose* сформирует новый столбец *purpose_category*, в который войдут следующие категории:

* 'операции с автомобилем',
* 'операции с недвижимостью',
* 'проведение свадьбы',
* 'получение образования'.

In [ ]:
df.to_sql('credits',con,index=False,if_exists='replace')

In [ ]:
sql = '''SELECT t.*,

CASE WHEN t.purpose LIKE '%авто%' OR t.purpose LIKE '%машин%' THEN 'операции с автомобилем'
WHEN t.purpose LIKE '%жиль%' OR t.purpose LIKE '%недвиж%' THEN 'операции с недвижимостью'
WHEN t.purpose LIKE '%свадьб%' THEN 'проведение свадьбы'
WHEN t.purpose LIKE '%образ%' THEN 'получение образования'
ELSE 'другое' END 
AS purpose_category

FROM credits t'''

In [ ]:
df = select(sql)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


# ИССЛЕДОВАНИЕ ДАННЫХ:


Есть ли зависимость между количеством детей и возвратом кредита в срок?

Приступаем к проверке первой гипотезы:


In [ ]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


Сделаем сводную таблицу:

In [ ]:
children_pivot = df.groupby('children')['debt'].agg(['count','mean'])

In [ ]:
children_pivot

,count,mean
children,,
0,14091,0.075438
1,4808,0.092346
2,2052,0.094542
3,330,0.081818
4,41,0.097561
5,9,0.000000


Для удобства и наглядности переведем столбец в проценты, переименуем и отсортируем:

In [ ]:
children_pivot['mean'] = children_pivot['mean'] * 100

In [ ]:
children_pivot = children_pivot.rename(columns={"mean": "debt_perc"})

In [ ]:
print(children_pivot.sort_values(by='debt_perc', ascending=False))

          count  debt_perc
children                  
4            41   9.756098
2          2052   9.454191
1          4808   9.234609
3           330   8.181818
0         14091   7.543822
5             9   0.000000


Некоторые выборки являются недотаточными для построения выводов, уберём их из сводной таблицы:

In [ ]:
rows = children_pivot['count']>=500
print(children_pivot[rows].sort_values(by='debt_perc', ascending=False))

          count  debt_perc
children                  
2          2052   9.454191
1          4808   9.234609
0         14091   7.543822


*Вывод:* 


Если убрать группы, которые являются слишком малочисленными для анализа, то мы видим такую картину:

*   Заемщики с числом детей *1* и *2* имеют схожий процент должников (*9.45%* и *9.23%*)
*   В группе *бездетных заемщиков* процент должников значительно ниже - *7.5%*




Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


In [ ]:
print(df['family_status'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


In [ ]:
print(df['family_status_id'].unique())

[0 1 2 3 4]


In [ ]:
rows = df['family_status_id']==3

In [ ]:
df[rows].head(1)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
19,0,10038.818549,48,среднее,1,в разводе,3,F,сотрудник,0,242831,на покупку своего автомобиля,B,операции с автомобилем


In [ ]:
rows = df['family_status_id']==4

In [ ]:
df[rows].head(1)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547,операции с коммерческой недвижимостью,B,операции с недвижимостью


Мы видим 5 вариантов семейного положения и 5 id семейного положения. На всякий случай я проверил, что последовательность совпадает (не был уверен, что это так)

In [ ]:
family_status_pivot = df.groupby('family_status')['debt'].agg(['count','mean'])
family_status_pivot['mean'] = family_status_pivot['mean'] * 100
family_status_pivot = family_status_pivot.rename(columns={"mean": "debt_perc"})

In [ ]:
print(family_status_pivot.sort_values(by='debt_perc', ascending=False))

                       count  debt_perc
family_status                          
Не женат / не замужем   2796   9.763948
гражданский брак        4134   9.313014
женат / замужем        12261   7.560558
в разводе               1189   7.064760
вдовец / вдова           951   6.624606


*Вывод:* 
Мы наблюдаем интересную картину. Наибольший процент должников находится в группе не женатых и находящихся в гражданском браке заемщиков, однако наименьший процент показывают группы заемщиков, находящихся в разводе или статусе вдовца/вдовы.
Кажется факт совмесного проживания с законным супругом мало что значит для надежности заёмщика.

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
df.to_sql('credits',con,index=False,if_exists='replace')

In [ ]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


In [ ]:
sql ='''
SELECT
t.total_income_category,

COUNT(*) AS cnt,
AVG(t.debt) * 100 AS debt_perc

 FROM credits t
 GROUP BY t.total_income_category
 ORDER BY AVG(t.debt) * 100 DESC
 '''
select(sql)

,total_income_category,cnt,debt_perc
0,E,22,9.090909
1,C,15921,8.498210
2,A,25,8.000000
3,B,5014,7.060231
4,D,349,6.017192


Опять же часть выборок слишком мала, убираем:

In [ ]:
income_cat_pivot = select(sql)
income_cat_pivot.to_sql('income_cat_pivot',con,index=False,if_exists='replace')
sql = ''' SELECT
t.* from income_cat_pivot t
where t.cnt > 500
'''
select(sql)


,total_income_category,cnt,debt_perc
0,C,15921,8.498210
1,B,5014,7.060231


*Вывод:*

К сожалению мы можем сравнить только две выборки и они показывают, что люди с более высокой категорией дохода (**B**) значительно реже задерживают платежи по кредитам (***7%***) , чем люди с более низким доходом (**C**) среди которых процент должников составляет ***8.5%***

Было бы интересно собрать больше данных о других группах заёмщиков или  посмотреть на средний размер займа и другие показатели, но в рамках данного исследования мы не распологаем такой информацией.

Как разные цели кредита влияют на его возврат в срок?

In [ ]:
purpose_category_pivot = df.groupby('purpose_category')['debt'].agg(['count','mean'])
purpose_category_pivot['mean'] = purpose_category_pivot['mean'] * 100
purpose_category_pivot = purpose_category_pivot.rename(columns={"mean": "debt_perc"})

In [ ]:
print(purpose_category_pivot.sort_values(by='debt_perc', ascending=False))

                          count  debt_perc
purpose_category                          
операции с автомобилем     4279   9.347978
получение образования      3988   9.252758
проведение свадьбы         2313   7.911803
операции с недвижимостью  10751   7.255139


*Вывод:*

Мы видим, что займы на *операции с автомобилем* и на *получение образования* имеют близкий процент должников (*9.35%* и *9.25%*), что ощутимо выше цифр по займам на *проведение свадьбы* и на *операции с недвижимостью*, которые в свою очередь тоже имеют близкие показатели(*7.91%* и *7.26%*) 

Учитывая, что у нас пропущенные значения встречаются в столбцах "стаж" и "общий доход" я думаю это может быть связано с тем, что некоторые клиенты на момент получения кредита ещё не имели официального трудоустройства и дохода, такое легко себе представить в случае со студентом, который берет образовательный кредит. Или для человека, который длительное время является безработным (тк мы не знаем за какой период рассчитывается доход)

# ОБЩИЙ ВЫВОД:

Мы исследовали надёжность заемщиков банка. В данных были пропущены значения в столбцах "days_employed" и "total_income". Пропуски мы заменили медианными значениями. Так же были обнаружены аномальными значением количества детей, количеством рабочих дней, а также явные и неявные дубликаты. Все ошибки были устранены. 


Мы категоризировали данные по уровню дохода и цели кредита

В ходе нашего небольшого исследования мы овтетили на различные вопросы:
*   **Есть ли зависимость между количеством детей и возвратом кредита в срок?**

На основании имеющихся данных мы пришли к выводу, что бездетные заемщики значительно реже задерживают платежи, а процент должников у групп заемщиков с 1 и 2 детьми отличается незначительно.

*   **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Мы обнаружили зависимость - не состоящие в браке или состоящие в гражданском браке заёмщики значительно чаще задерживают платежи по кредиту, чем заемщики находящиеся в браке, в разводе и являющие вдовцами/вдовами

* **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Мы выяснили, что люди с более высоким доходом реже становятся должниками, чем люди с более низким доходом, однако достаточными для анализа оказались только 2 группы из 5.

*   **Как разные цели кредита влияют на его возврат в срок?**

Нам удалось выяснить, что цель кредита значительно влияет на вероятность его возврата в срок. Цели кредита из нашего исследования можно условно разделить на две группы - **"свадьбы и недвижимость"**, которая имеет значительно более высокий процент должников, чем группа **"автомобили и образование"**.